<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Merge-IBES-and-CRSP" data-toc-modified-id="Merge-IBES-and-CRSP-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merge IBES and CRSP</a></span></li><li><span><a href="#Add-CIK" data-toc-modified-id="Add-CIK-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add CIK</a></span></li></ul></div>

In [1]:
# Objective of this code is to add the CRSP after-hours returns and one month ahead returns.

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime, time
bday_us = CustomBusinessDay(calendar=USFederalHolidayCalendar())

# Load data

In [14]:
# IBES
# This code loads the data for the IBES SUE constructed for the CFR RIP paper.
# The code is the IBES_Data_Processing_1983_2019
ibes = pd.read_hdf('../../Data/IBES/Earnings_Surprises_1983_2019_v01.h5')

# keep only the earnings announcements for which we have timestamps
ibes = ibes[ibes['IBES_Timestamp'].dt.time != time(0)]
ibes = ibes[ibes['IBES_Timestamp'].dt.year >= 1996]

In [41]:
# CRSP
crsp = pd.read_csv('../../Data/CRSP/crsp_dsf_1970_2019_sc10_11_ec123.csv.gz', parse_dates=['date'])
crsp = crsp[crsp['date'].dt.year>=1996]
crsp = crsp[['PERMNO', 'TICKER', 'date',  'PRC', 'CFACPR', 'OPENPRC', 'SHROUT', 'VOL', 'RET']]
crsp['PRC'] = np.abs(crsp.PRC)
crsp['OPENPRC'] = np.abs(crsp['OPENPRC'])
crsp['mcap'] = crsp['SHROUT']*crsp['PRC']

# adjust the closing price for splits on the next trading day (if it occurs)
crsp['LAGPRC'] = crsp.groupby('PERMNO')['PRC'].shift()
crsp['LAGPRC'] = np.where((crsp['CFACPR'].shift() != crsp['CFACPR']) &
                          (crsp['CFACPR'].shift() > 1), crsp['LAGPRC']/(crsp['CFACPR'].shift()/crsp['CFACPR']),
                          crsp['LAGPRC'])
crsp['PRC1'] = crsp.groupby('PERMNO')['PRC'].shift(-1)
crsp['OCRET'] = (crsp['PRC1']-crsp['OPENPRC'])/crsp['OPENPRC']

# Calculate the overnight return
crsp['ONRET'] = (crsp['OPENPRC']-crsp['LAGPRC'])/crsp['LAGPRC']

In [46]:
crsp['RET'] = np.where(crsp.RET.isin(['A', 'B', 'C']), np.nan, crsp.RET)
crsp['RET'] = crsp['RET'].astype(float)

In [42]:
# Merge the market returns from the fama french file
ff = pd.read_csv('../../Data/FamaFrench/F-F_Research_Data_Factors_daily.CSV')
ff['date'] = pd.to_datetime(ff['date'], format='%Y%m%d')
ff = ff[['date', 'Mkt-RF', 'RF']]
ff[['Mkt-RF', 'RF']] = ff[['Mkt-RF', 'RF']]/100
ff['Mkt'] = ff['Mkt-RF']-ff['RF']
ff = ff.rename(columns={'Mkt-RF':'ExMkt'})

In [43]:
crsp = pd.merge(crsp, ff, on=['date'])

In [44]:
crsp = crsp.sort_values(['PERMNO','date'], ascending=[True, False])

In [47]:
crsp['lnRET'] = np.log(1+crsp['RET'])
crsp['lnExMkt'] = np.log(1+crsp['ExMkt'])

crsp[['lnRET_2','lnExMkt_2']] = crsp.groupby('PERMNO')[['lnRET','lnExMkt']].rolling(2).sum().reset_index(0, drop=True)
crsp[['lnRET_30','lnExMkt_30']] = crsp.groupby('PERMNO')[['lnRET','lnExMkt']].rolling(30).sum().reset_index(0, drop=True)
crsp[['lnRET_60','lnExMkt_60']] = crsp.groupby('PERMNO')[['lnRET','lnExMkt']].rolling(60).sum().reset_index(0, drop=True)

In [49]:
crsp['BHAR_0_1'] = crsp['lnRET_2']-crsp['lnExMkt_2']

In [50]:
crsp['BHAR_2_30'] = np.exp(crsp['lnRET_30'] - crsp['lnRET_2']) - np.exp(crsp['lnExMkt_30'] - crsp['lnExMkt_2'])
crsp['BHAR_2_60'] = np.exp(crsp['lnRET_60'] - crsp['lnRET_2']) - np.exp(crsp['lnExMkt_60'] - crsp['lnExMkt_2'])

In [51]:
crsp = crsp.sort_values(['PERMNO','date'])

In [57]:
# adjust date if earnings after 4 pm
ibes['IBES_Date_Adj_ON'] = np.where(ibes['IBES_Timestamp'].dt.time>=time(16),
                                    ibes['date']+pd.offsets.Day(1), ibes['date'])

In [58]:
trading_days = pd.DataFrame(index=(np.sort(list(crsp['date'].dt.date.unique()))))

In [59]:
# (3) Create an IBES Adjusted date to get the first trading day following the
# news if the EA is on a holiday or weekend.
date_adj = []
for x in ibes['IBES_Date_Adj_ON'].dt.date:
    if x in trading_days:
        date_adj.append(x)
    else:
        date_adj.append(trading_days.loc[x:].index.min())

In [60]:
ibes['date'] = date_adj

In [66]:
ibes = ibes[['date', 'PERMNO','GVKEY', 'CNAME',  'IBES_Timestamp',  'Year', 'Quarter', 'rdq',
             'SUE_Med', 'sue_rw', 'ACTUAL', 'ADJ_ACTUAL', 'numest', 'dispersion',
             'ggroup', 'gind', 'gsector', 'gsubind', 'medest',  'sp500', 'sp400', 'sp600', 'sp1500']]

# Merge IBES and CRSP

In [71]:
ibes['date'] = pd.to_datetime(ibes['date'])

In [72]:
ibes_merged = pd.merge(ibes, crsp, on=['date', 'PERMNO'], how='left')

In [74]:
ibes_merged.to_hdf('../../Data/IBES_CRSP_Merged_1996_2019_v01.h5', key='panel')

# Add CIK

In [27]:
ibes_merged = pd.read_hdf('../../Data/IBES_CRSP_Merged_1996_2019_v01.h5')
ibes_merged['PERMNO'] = ibes_merged['PERMNO'].astype(int)
ibes_merged['GVKEY'] = ibes_merged['GVKEY'].astype(int)

In [25]:
cik = pd.read_csv('../../Data/CRSP/crsp_cik_merge.csv.gz').rename(columns={'gvkey':'GVKEY',
                                                                           'LPERMNO':"PERMNO",
                                                                           'cik':"CIK", 'LINKDT':'date'})
cik = cik[cik['CIK'].notnull()]
cik = cik[['GVKEY', 'PERMNO', 'CIK', 'date']]
cik['date'] = pd.to_datetime(cik['date'])
cik['CIK'] = cik['CIK'].astype(int)
cik['PERMNO'] = cik['PERMNO'].astype(int)
cik['GVKEY'] = cik['GVKEY'].astype(int)


In [31]:
ibes_merged = ibes_merged.sort_values(by='date')
cik = cik.sort_values(by='date')

In [35]:
ibes_merged = pd.merge_asof(ibes_merged, cik, on='date', by=['PERMNO', 'GVKEY'])

In [37]:
ibes_merged.to_hdf('../../Data/IBES_CRSP_Merged_1996_2019_v01.h5', key='panel')